#v.0

In [ ]:
import math
from functools import reduce
from operator import mul

def lcm(a: int, b: int) -> int:
    """Least common multiple."""
    return abs(a * b) // math.gcd(a, b) if a and b else 0

def compute_affinity(m1: dict, m2: dict) -> float:
    """
    Compute Clarence Barlow’s metrical affinity M between two meters.

    Each meter dict must have:
      - 'q': list of stratification divisors (q1, q2, …)
      - 'v': bar‐tempo (an integer)
      - 'psi': list of indispensability values for each original pulse

    Steps:
      1. Ω_u = product of q’s (original pulse‐count per bar)
      2. p  = v * Ω_u           (original pulses per minute)
      3. c  = lcm(p1, p2)      (common pulse tempo)
      4. Ω_z = c // v          (extended pulse‐count per bar)
      5. Ω₀  = lcm(Ω_z1, Ω_z2) (whole‐cycle length)
      6. Sum over n=1…Ω₀ of [ψ₁(n)·ψ₂(n)]², where ψᵢ(n) = psiᵢ[(n−1) mod Ω_uᵢ]
      7. Plug into:
         M = -1 / (2 * ln( [18·S - 2] / [7·Ω₀·(Ω_z1−1)²·(Ω_z2−1)²] ))
    """
    q1, v1, psi1 = m1['q'], m1['v'], m1['psi']
    q2, v2, psi2 = m2['q'], m2['v'], m2['psi']

    # 1. original stratification depths
    Omega_u1 = reduce(mul, q1, 1)
    Omega_u2 = reduce(mul, q2, 1)
    if len(psi1) != Omega_u1:
        raise ValueError(f"len(psi1)={len(psi1)} but product(q1)={Omega_u1}")
    if len(psi2) != Omega_u2:
        raise ValueError(f"len(psi2)={len(psi2)} but product(q2)={Omega_u2}")

    # 2. original pulses/minute
    p1 = v1 * Omega_u1
    p2 = v2 * Omega_u2

    # 3. common pulse tempo
    c = lcm(p1, p2)

    # 4. extended stratifications
    Omega_z1 = c // v1
    Omega_z2 = c // v2

    # 5. full non‐repeating cycle length
    Omega_0 = lcm(Omega_z1, Omega_z2)

    # 6. sum of squared products
    S = 0
    for n in range(Omega_0):
        # map n→ original‐psi index via modulo Ω_u
        x1 = psi1[n % Omega_u1]
        x2 = psi2[n % Omega_u2]
        S += (x1 * x2) ** 2

    # 7. final formula
    numerator   = 18 * S - 2
    denominator = 7 * Omega_0 * ( (Omega_z1 - 1) ** 2 ) * ( (Omega_z2 - 1) ** 2 )
    ratio       = numerator / denominator

    if ratio <= 0:
        raise ValueError("Log undefined for non‐positive argument; check inputs.")
    M = -1.0 / (2 * math.log(ratio))
    return M

if __name__ == "__main__":
    # === Test cases ===
    tests = [
        {
            # Test 0: 2×2×3@MM20 vs 3×5@MM16 → M ≈ 0.15730 (often off in many implementations)
            'm1': { 'q':[2,2,3], 'v':20, 'psi':[11,0,4,8,2,6,10,1,5,9,3,7] },
            'm2': { 'q':[3,5],   'v':16, 'psi':[14,0,9,3,6,12,1,10,4,7,13,2,11,5,8] }
        },
        {
            # Test 1: 3×2@1 vs 2×3@1 → M ≈ 0.32450
            'm1': { 'q':[3,2],   'v':1, 'psi':[5,0,3,1,4,2] },
            'm2': { 'q':[2,3],   'v':1, 'psi':[5,0,2,4,1,3] }
        },
        {
            # Test 2: 3×2×2@1 vs 2×3×2@1 → M ≈ 0.36421
            'm1': { 'q':[3,2,2], 'v':1, 'psi':[11,0,6,3,9,1,7,4,10,2,8,5] },
            'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] }
        },
        {
            # Test 3: 2×2×2@1 vs 2×3×2@1 → M ≈ 0.18956 (usually fails)
            'm1': { 'q':[2,2,2], 'v':1, 'psi':[7,0,4,2,6,1,5,3] },
            'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] }
        }
    ]

    for i, t in enumerate(tests):
        try:
            M = compute_affinity(t['m1'], t['m2'])
            print(f"Test {i}: M = {M:.5f}")
        except Exception as e:
            print(f"Test {i}: error ({e})")

Test 0: M = 0.44779
Test 1: M = 0.72095
Test 2: M = 0.80925
Test 3: M = 0.11127


# v.1

In [ ]:
import math
from functools import reduce
from operator import mul
import pandas as pd
!pip install ace_tools_open
from ace_tools_open import display_dataframe_to_user


# Least common multiple helper
def lcm(a, b):
    return abs(a * b) // math.gcd(a, b) if a and b else 0

# Test cases as provided
tests = [
    {
        'name': 'Test 0',
        'm1': { 'q':[2,2,3], 'v':20, 'psi':[11,0,4,8,2,6,10,1,5,9,3,7] },
        'm2': { 'q':[3,5],   'v':16, 'psi':[14,0,9,3,6,12,1,10,4,7,13,2,11,5,8] },
    },
    {
        'name': 'Test 1',
        'm1': { 'q':[3,2],   'v':1, 'psi':[5,0,3,1,4,2] },
        'm2': { 'q':[2,3],   'v':1, 'psi':[5,0,2,4,1,3] },
    },
    {
        'name': 'Test 2',
        'm1': { 'q':[3,2,2], 'v':1, 'psi':[11,0,6,3,9,1,7,4,10,2,8,5] },
        'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] },
    },
    {
        'name': 'Test 3',
        'm1': { 'q':[2,2,2], 'v':1, 'psi':[7,0,4,2,6,1,5,3] },
        'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] },
    }
]

# Variants to try:
# - numerator formula: '18S-2' or '9S-1'
# - Omega_z calculation: 'div_v' (c//v) or 'full_c' (c)
variants = [
    ('18S-2 & Omega_z=c//v', '18S-2', 'div_v'),
    ('9S-1 & Omega_z=c//v', '9S-1', 'div_v'),
    ('18S-2 & Omega_z=c',     '18S-2', 'full_c'),
    ('9S-1 & Omega_z=c',      '9S-1', 'full_c'),
]

# Collect results
results = []
for var_name, num_type, omega_type in variants:
    row = {'Variant': var_name}
    for test in tests:
        m1, m2 = test['m1'], test['m2']
        # Original stratification depths
        Omega_u1 = reduce(mul, m1['q'], 1)
        Omega_u2 = reduce(mul, m2['q'], 1)
        # Pulse tempos
        p1 = m1['v'] * Omega_u1
        p2 = m2['v'] * Omega_u2
        # Common pulse tempo
        c = lcm(p1, p2)
        # Extended stratification depths:
        if omega_type == 'div_v':
            Omega_z1 = c // m1['v']
            Omega_z2 = c // m2['v']
        else:  # full cycle c for both
            Omega_z1 = c
            Omega_z2 = c
        # Full non-repeating cycle
        Omega_0 = lcm(Omega_z1, Omega_z2)
        # Summation
        S = sum((m1['psi'][n % Omega_u1] * m2['psi'][n % Omega_u2])**2 for n in range(Omega_0))
        # Numerator choices
        if num_type == '18S-2':
            numerator = 18 * S - 2
        else:
            numerator = 9 * S - 1
        # Denominator
        denominator = 7 * Omega_0 * ((Omega_z1 - 1)**2) * ((Omega_z2 - 1)**2)
        ratio = numerator / denominator
        # Compute M
        try:
            M = -1.0 / (2 * math.log(ratio))
        except ValueError:
            M = float('nan')
        row[test['name']] = round(M, 5)
    results.append(row)

# Display results in a table for comparison
df = pd.DataFrame(results)
display_dataframe_to_user('Affinity Variants Comparison', df)
print('Expected values: Test 0 = 0.15730, Test 1 = 0.32450, Test 2 = 0.36421, Test 3 = 0.18956')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.2 MB/s eta 0:00:00
Affinity Variants Comparison


Expected values: Test 0 = 0.15730, Test 1 = 0.32450, Test 2 = 0.36421, Test 3 = 0.18956


In [ ]:
import math
from functools import reduce
from operator import mul

# Helper for least common multiple
def lcm(a, b):
    return abs(a * b) // math.gcd(a, b) if a and b else 0

# Test definitions
tests = [
    {
        'name': 'Test 0',
        'm1': { 'q':[2,2,3], 'v':20, 'psi':[11,0,4,8,2,6,10,1,5,9,3,7] },
        'm2': { 'q':[3,5],   'v':16, 'psi':[14,0,9,3,6,12,1,10,4,7,13,2,11,5,8] },
    },
    {
        'name': 'Test 1',
        'm1': { 'q':[3,2],   'v':1, 'psi':[5,0,3,1,4,2] },
        'm2': { 'q':[2,3],   'v':1, 'psi':[5,0,2,4,1,3] },
    },
    {
        'name': 'Test 2',
        'm1': { 'q':[3,2,2], 'v':1, 'psi':[11,0,6,3,9,1,7,4,10,2,8,5] },
        'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] },
    },
    {
        'name': 'Test 3',
        'm1': { 'q':[2,2,2], 'v':1, 'psi':[7,0,4,2,6,1,5,3] },
        'm2': { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] },
    }
]

# Variants: (description, numerator type, Omega_z type)
variants = [
    ('18S-2 & Omega_z=c//v', '18S-2', 'div_v'),
    ('9S-1 & Omega_z=c//v', '9S-1', 'div_v'),
    ('18S-2 & Omega_z=c',     '18S-2', 'full_c'),
    ('9S-1 & Omega_z=c',      '9S-1', 'full_c'),
]

# Perform computations
results = []
for var_name, num_type, omega_type in variants:
    row = {'Variant': var_name}
    for test in tests:
        m1, m2 = test['m1'], test['m2']
        # Original stratification
        Omega_u1 = reduce(mul, m1['q'], 1)
        Omega_u2 = reduce(mul, m2['q'], 1)
        # Pulse tempos
        p1 = m1['v'] * Omega_u1
        p2 = m2['v'] * Omega_u2
        # Common pulse tempo
        c = lcm(p1, p2)
        # Extended stratification
        if omega_type == 'div_v':
            Omega_z1 = c // m1['v']
            Omega_z2 = c // m2['v']
        else:
            Omega_z1 = c
            Omega_z2 = c
        # Full cycle length
        Omega_0 = lcm(Omega_z1, Omega_z2)
        # Summation of squared products
        S = sum((m1['psi'][n % Omega_u1] * m2['psi'][n % Omega_u2])**2 for n in range(Omega_0))
        # Numerator choice
        numerator = 18 * S - 2 if num_type == '18S-2' else 9 * S - 1
        # Denominator
        denominator = 7 * Omega_0 * ((Omega_z1 - 1)**2) * ((Omega_z2 - 1)**2)
        # Compute M
        ratio = numerator / denominator
        M = -1.0 / (2 * math.log(ratio)) if ratio > 0 else float('nan')
        row[test['name']] = round(M, 5)
    results.append(row)

# Display results
print(f"{'Variant':<25} {'Test0':<8} {'Test1':<8} {'Test2':<8} {'Test3':<8}")
for r in results:
    print(f"{r['Variant']:<25} {r['Test 0']:<8} {r['Test 1']:<8} {r['Test 2']:<8} {r['Test 3']:<8}")
print("\nExpected: Test 0 = 0.15730, Test 1 = 0.32450, Test 2 = 0.36421, Test 3 = 0.18956")


Variant                   Test0    Test1    Test2    Test3   
18S-2 & Omega_z=c//v      0.44779  0.72095  0.80925  0.11127 
9S-1 & Omega_z=c//v       0.27628  0.36057  0.38139  0.0964  
18S-2 & Omega_z=c         0.03861  0.72095  0.80925  0.11127 
9S-1 & Omega_z=c          0.03665  0.36057  0.38139  0.0964  

Expected: Test 0 = 0.15730, Test 1 = 0.32450, Test 2 = 0.36421, Test 3 = 0.18956


#v.6

In [ ]:
import math
from functools import reduce
from operator import mul

def lcm(a: int, b: int) -> int:
    """Compute least common multiple of a and b."""
    return abs(a * b) // math.gcd(a, b) if a and b else 0

def prime_factors(n: int) -> list:
    """Return the prime factorization of n as a list of prime factors."""
    factors = []
    # Handle factor 2
    while n % 2 == 0:
        factors.append(2)
        n //= 2
    # Handle odd factors
    p = 3
    while p * p <= n:
        while n % p == 0:
            factors.append(p)
            n //= p
        p += 2
    # If remainder is prime
    if n > 1:
        factors.append(n)
    return factors

def compute_affinity(m1: dict, m2: dict) -> float:
    """
    Compute Clarence Barlow’s metrical affinity using prime-factor extension
    and relative indispensability scaling.

    Each meter dict must have:
      - 'q': list of stratification divisors
      - 'v': bar-tempo (MM)
      - 'psi': list of indispensability values
    """
    # 1. Original stratification depths
    Omega_u1 = reduce(mul, m1['q'], 1)
    Omega_u2 = reduce(mul, m2['q'], 1)

    # 2. Pulse tempos
    p1 = m1['v'] * Omega_u1
    p2 = m2['v'] * Omega_u2

    # 3. Common pulse tempo
    c = lcm(p1, p2)

    # 4. Extension ratios
    r1 = c // p1
    r2 = c // p2

    # 5. Prime-factor extension
    ext1 = sorted(prime_factors(r1), reverse=True)
    ext2 = sorted(prime_factors(r2), reverse=True)
    print(f"Prime-factor extension: {ext1} and {ext2}")

    # 6. Extended stratifications
    q_ext1 = m1['q'] + ext1
    q_ext2 = m2['q'] + ext2
    Omega_z1 = reduce(mul, q_ext1, 1)
    Omega_z2 = reduce(mul, q_ext2, 1)

    # 7. Full non-repeating cycle length
    Omega_0 = lcm(Omega_z1, Omega_z2)

    # 8. Summation over full cycle using relative psi
    S = 0.0
    for n in range(Omega_0):
        psi1 = m1['psi'][n % Omega_u1] / (Omega_u1 - 1)
        psi2 = m2['psi'][n % Omega_u2] / (Omega_u2 - 1)
        S += (psi1 * psi2) ** 2

    # 9. Mean pulse similarity (MPS)
    MPS = S / Omega_0

    # 10. Final metrical affinity
    ratio = (9 * MPS - 1) / 3.5
    if ratio <= 0:
        raise ValueError("Log argument non-positive; check inputs")
    M = -1.0 / (2 * math.log(ratio))
    return M

if __name__ == "__main__":
    # Test cases
    tests = [
        ('Test 0', { 'q':[2,2,3], 'v':20, 'psi':[11,0,4,8,2,6,10,1,5,9,3,7] },
                   { 'q':[3,5],   'v':16, 'psi':[14,0,9,3,6,12,1,10,4,7,13,2,11,5,8] }),
        ('Test 1', { 'q':[3,2],   'v':1, 'psi':[5,0,3,1,4,2] },
                   { 'q':[2,3],   'v':1, 'psi':[5,0,2,4,1,3] }),
        ('Test 2', { 'q':[3,2,2], 'v':1, 'psi':[11,0,6,3,9,1,7,4,10,2,8,5] },
                   { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] }),
        ('Test 3', { 'q':[2,2,2], 'v':1, 'psi':[7,0,4,2,6,1,5,3] },
                   { 'q':[2,3,2], 'v':1, 'psi':[11,0,6,2,8,4,10,1,7,3,9,5] }),
    ]

    for name, m1, m2 in tests:
        try:
            M = compute_affinity(m1, m2)
            print(f"{name}: M = {M:.5f}")
        except Exception as e:
            print(f"{name}: Error - {e}")

    print("\nExpected: Test 0 = 0.1573, Test 1 = 0.3245, Test 2 = 0.36421, Test 3 = 0.18956")


Prime-factor extension: [] and []
Test 0: M = 0.15734
Prime-factor extension: [] and []
Test 1: M = 0.32447
Prime-factor extension: [] and []
Test 2: M = 0.36421
Prime-factor extension: [] and []
Test 3: M = 0.35240

Expected: Test 0 = 0.1573, Test 1 = 0.3245, Test 2 = 0.36421, Test 3 = 0.18956


#v.6.5

In [ ]:
import math
from functools import reduce
from operator import mul

# Helper functions
def lcm(a: int, b: int) -> int:
    return abs(a * b) // math.gcd(a, b) if a and b else 0

def prime_factors(n: int) -> list:
    factors = []
    # factor out 2
    while n % 2 == 0:
        factors.append(2)
        n //= 2
    # odd factors
    p = 3
    while p * p <= n:
        while n % p == 0:
            factors.append(p)
            n //= p
        p += 2
    if n > 1:
        factors.append(n)
    return sorted(factors, reverse=True)

def mod(x: int, m: int) -> int:
    return (x % m + m) % m

# Forward declaration for mutual recursion
def calculate_indispensability(prime_factors_list):
    pass

def basic_psi(p: int, n: int) -> int:
    # Case for p == 2
    if p == 2:
        return 2 - n
    # Case for n == p - 1
    if n == p - 1:
        return math.floor(p / 4)
    # recursive case
    index = n - (n // p)
    factors = prime_factors(p - 1)
    series = calculate_indispensability(factors)
    q = series[index - 1]
    return math.floor(q + 2 * math.sqrt((q + 1) / p))

def calculate_indispensability(prime_factors_list: list) -> list:
    z = len(prime_factors_list)
    extended = [1] + prime_factors_list + [1]
    total_pulses = reduce(mul, prime_factors_list, 1)
    mod_base = total_pulses
    results = []
    for n in range(1, total_pulses + 1):
        psi = 0
        for r in range(z):
            # multiplier = product of extended[0 .. z-r-1]
            multiplier = reduce(mul, extended[:z-r], 1)
            # Q = product of extended[z+1-k] for k=0..r
            Q = reduce(mul, (extended[z+1-k] for k in range(r+1)), 1)
            A = mod(n - 2, mod_base)
            quotient = math.floor(1 + (A / Q))
            p_current = extended[z - r]
            D = mod(quotient, p_current)
            psi += multiplier * basic_psi(p_current, 1 + D)
        results.append(psi)
    return results

def compute_affinity(m1: dict, m2: dict) -> float:
    # original stratification depths
    Omega_u1 = reduce(mul, m1['q'], 1)
    Omega_u2 = reduce(mul, m2['q'], 1)
    # pulse tempos
    p1 = m1['v'] * Omega_u1
    p2 = m2['v'] * Omega_u2
    # common pulse tempo
    c = lcm(p1, p2)
    # extension ratios
    r1, r2 = c // p1, c // p2
    # extend stratifications
    ext1, ext2 = prime_factors(r1), prime_factors(r2)
    q_ext1 = m1['q'] + ext1
    q_ext2 = m2['q'] + ext2
    Omega_z1 = reduce(mul, q_ext1, 1)
    Omega_z2 = reduce(mul, q_ext2, 1)
    # build extended indispensability arrays
    psi_z1 = calculate_indispensability(q_ext1)
    psi_z2 = calculate_indispensability(q_ext2)
    # full cycle length
    Omega_0 = lcm(Omega_z1, Omega_z2)
    # sum over cycle
    S = 0.0
    for n in range(Omega_0):
        rel1 = psi_z1[n % Omega_z1] / (Omega_z1 - 1)
        rel2 = psi_z2[n % Omega_z2] / (Omega_z2 - 1)
        S += (rel1 * rel2) ** 2
    MPS = S / Omega_0
    ratio = (9 * MPS - 1) / 3.5
    if ratio <= 0:
        raise ValueError("Non-positive log argument")
    return -1.0 / (2 * math.log(ratio))

# Test cases
tests = [
    ('Test 0', { 'q':[2,2,3], 'v':20, 'psi':[] }, { 'q':[3,5], 'v':16, 'psi':[] }),
    ('Test 1', { 'q':[3,2], 'v':1, 'psi':[] },   { 'q':[2,3],  'v':1, 'psi':[] }),
    ('Test 2', { 'q':[3,2,2], 'v':1, 'psi':[] }, { 'q':[2,3,2],'v':1, 'psi':[] }),
    ('Test 3', { 'q':[2,2,2], 'v':1, 'psi':[] }, { 'q':[2,3,2],'v':1, 'psi':[] })
]

for name, m1, m2 in tests:
    M = compute_affinity(m1, m2)
    print(f"{name}: M = {M:.5f}")
print("\nExpected: Test 0 = 0.1573, Test 1 = 0.3245, Test 2 = 0.36421, Test 3 = 0.18956")


Test 0: M = 0.15734
Test 1: M = 0.32447
Test 2: M = 0.36421
Test 3: M = 0.18956

Expected: Test 0 = 0.1573, Test 1 = 0.3245, Test 2 = 0.36421, Test 3 = 0.18956


#v.6.6

In [1]:
import math
from functools import reduce
from operator import mul
import pandas as pd
!pip install ace_tools_open
from ace_tools_open import display_dataframe_to_user

# Helper functions
def lcm(a: int, b: int) -> int:
    return abs(a * b) // math.gcd(a, b) if a and b else 0

def prime_factors(n: int) -> list:
    factors = []
    while n % 2 == 0:
        factors.append(2); n //= 2
    p = 3
    while p * p <= n:
        while n % p == 0:
            factors.append(p); n //= p
        p += 2
    if n > 1:
        factors.append(n)
    return sorted(factors, reverse=True)

def mod(x: int, m: int) -> int:
    return (x % m + m) % m

def basic_psi(p: int, n: int) -> int:
    if p == 2:
        return 2 - n
    if n == p - 1:
        return math.floor(p / 4)
    index = n - (n // p)
    series = calculate_indispensability(prime_factors(p - 1))
    q = series[index - 1]
    return math.floor(q + 2 * math.sqrt((q + 1) / p))

def calculate_indispensability(q_list: list) -> list:
    z = len(q_list)
    extended = [1] + q_list + [1]
    total_pulses = reduce(mul, q_list, 1)
    results = []
    for n in range(1, total_pulses + 1):
        psi = 0
        for r in range(z):
            multiplier = reduce(mul, extended[:z-r], 1)
            Q = reduce(mul, (extended[z+1-k] for k in range(r+1)), 1)
            A = mod(n - 2, total_pulses)
            quotient = math.floor(1 + (A / Q))
            p_current = extended[z - r]
            D = mod(quotient, p_current)
            psi += multiplier * basic_psi(p_current, 1 + D)
        results.append(psi)
    return results

def compute_affinity(m1: dict, m2: dict) -> float:
    Omega_u1 = reduce(mul, m1['q'], 1)
    Omega_u2 = reduce(mul, m2['q'], 1)
    p1 = m1['v'] * Omega_u1
    p2 = m2['v'] * Omega_u2
    c = lcm(p1, p2)
    r1, r2 = c // p1, c // p2
    ext1, ext2 = prime_factors(r1), prime_factors(r2)
    q_ext1 = m1['q'] + ext1
    q_ext2 = m2['q'] + ext2
    Omega_z1 = reduce(mul, q_ext1, 1)
    Omega_z2 = reduce(mul, q_ext2, 1)
    psi_z1 = calculate_indispensability(q_ext1)
    psi_z2 = calculate_indispensability(q_ext2)
    Omega_0 = lcm(Omega_z1, Omega_z2)
    S = 0.0
    for n in range(Omega_0):
        rel1 = psi_z1[n % Omega_z1] / (Omega_z1 - 1)
        rel2 = psi_z2[n % Omega_z2] / (Omega_z2 - 1)
        S += (rel1 * rel2) ** 2
    MPS = S / Omega_0
    ratio = (9 * MPS - 1) / 3.5
    if ratio <= 0:
        return float('nan')
    return -1.0 / (2 * math.log(ratio))

def test_trimmed_table8():
    # Trimmed rows for meters 2x2x2 and 2x3x2 with various v
    tests = [
        (1, [2,3,2], 1, [2,3,2], 0.41454),
        (1, [2,3,2], 3, [2,2,2], 0.39582),
        (3, [2,2,2], 1, [2,3,2], 0.39582),
        (2, [2,3,2], 3, [2,2,2], 0.3524),
        (3, [2,2,2], 2, [2,3,2], 0.3524),
        (1, [2,2,2], 2, [2,3,2], 0.25708),
        (2, [2,3,2], 1, [2,2,2], 0.25708),
        (1, [2,2,2], 1, [2,3,2], 0.18956),
        (1, [2,3,2], 1, [2,2,2], 0.18956),
        (1, [2,3,2], 2, [2,2,2], 0.16281),
        (2, [2,2,2], 1, [2,3,2], 0.16281),
        (1, [2,2,2], 3, [2,3,2], 0.1383),
        (3, [2,3,2], 1, [2,2,2], 0.1383),
        (2, [2,2,2], 3, [2,3,2], 0.11317),
        (3, [2,3,2], 2, [2,2,2], 0.11317),
    ]
    rows = []
    for v1, q1, v2, q2, expected in tests:
        m1 = {'q': q1, 'v': v1}
        m2 = {'q': q2, 'v': v2}
        computed = round(compute_affinity(m1, m2), 5)
        rows.append({
            'v1': v1,
            'q1': 'x'.join(map(str, q1)),
            'v2': v2,
            'q2': 'x'.join(map(str, q2)),
            'Computed M': computed,
            'Expected M': expected
        })
    df = pd.DataFrame(rows)
    display_dataframe_to_user('Trimmed Table 8 Results', df)


def test_table8():
    """
    Generate the full Table 8: metrical affinities for all stratifications up to 3 levels
    (2x2x2, 2x2x3, 2x3x2, 3x2x2) at bar‑tempos 1–3.
    """
    # Define the stratifications of order 0–3 that appear in Table 8
    strat_defs = {
        '2x2x2': [2, 2, 2],
        '2x2x3': [2, 2, 3],
        '2x3x2': [2, 3, 2],
        '3x2x2': [3, 2, 2],
    }
    tempos = [1, 2, 3]

    rows = []
    for v1 in tempos:
        for name1, q1 in strat_defs.items():
            m1 = {'q': q1, 'v': v1}
            for v2 in tempos:
                for name2, q2 in strat_defs.items():
                    m2 = {'q': q2, 'v': v2}
                    M = compute_affinity(m1, m2)
                    rows.append({
                        'v1': v1,
                        'q1': name1,
                        'v2': v2,
                        'q2': name2,
                        'M (computed)': round(M, 5)
                    })
    df = pd.DataFrame(rows)
    display_dataframe_to_user('Complete Table 8 Computations', df)

# Run the full Table 8 test
test_table8()


# Execute the tests
#test_trimmed_table8()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00
Complete Table 8 Computations


Loading ITables v2.4.0 from the internet... (need help?)


In [7]:
def test_pulseSync():
    # Trimmed rows for meters 2x2x2 and 2x3x2 with various v
    tests = [
        (1, [2,2,2,2], 1, [2,2,2,2], '2/2'),
        (1, [2,2,2], 1, [2,2,2,2], '2/4'),
        (1, [3,2,2], 1, [2,2,2,2], '3/4'),
        (1, [5,2], 1, [2,2,2,2], '5/8'),
        (1, [3,3,2], 1, [2,2,2,2], '9/8'),
        (1, [3,3,2], 1, [2,2,2,2], '9/8'),
        (1, [2,2,3,2], 1, [2,2,2,2], '12/8'),
        (1, [3,3,2,2], 1, [2,2,2,2], '12/4'),
    ]
    rows = []
    for v1, q1, v2, q2, meter in tests:
        m1 = {'q': q1, 'v': v1}
        m2 = {'q': q2, 'v': v2}
        computed = round(compute_affinity(m1, m2), 5)
        rows.append({
            'v1': v1,
            'q1': 'x'.join(map(str, q1)),
            'v2': v2,
            'q2': 'x'.join(map(str, q2)),
            'Computed M': computed,
            'Meter': meter
        })
    df = pd.DataFrame(rows)
    #display_dataframe_to_user('pulseSync Results', df)
    display(df)


# Execute the tests
test_pulseSync()

,v1,q1,v2,q2,Computed M,Meter
0,1,2x2x2x2,1,2x2x2x2,0.39318,2/2
1,1,2x2x2,1,2x2x2x2,0.39318,2/4
2,1,3x2x2,1,2x2x2x2,0.15083,3/4
3,1,5x2,1,2x2x2x2,0.10989,5/8
4,1,3x3x2,1,2x2x2x2,0.09837,9/8
5,1,3x3x2,1,2x2x2x2,0.09837,9/8
6,1,2x2x3x2,1,2x2x2x2,0.18045,12/8
7,1,3x3x2x2,1,2x2x2x2,0.09837,12/4


In [ ]:
def test_beatSync():
    # Trimmed rows for meters 2x2x2 and 2x3x2 with various v
    tests = [
        (1, [2,2,2,2], 1, [2,2,2,2], '4/4'),
        (1, [2,2,3,2,2], 1, [2,2,2,2], '12/4'),
        (1, [2,2,3,2], 1, [2,2,2,2], '12/8'),
        (1, [2,3,2], 1, [2,2,2,2], '6/8'),
        (1, [3,2,2], 1, [2,2,2,2], '3/4'),
        (1, [2,3,2,2], 1, [2,2,2,2], '6/4'),
        (1, [3,3,2,2], 1, [2,2,2,2], '9/4'),
        (1, [7,2,2], 1, [2,2,2,2], '7/4'),
    ]
    rows = []
    for v1, q1, v2, q2, meter in tests:
        m1 = {'q': q1, 'v': v1}
        m2 = {'q': q2, 'v': v2}
        computed = round(compute_affinity(m1, m2), 5)
        rows.append({
            'v1': v1,
            'q1': 'x'.join(map(str, q1)),
            'v2': v2,
            'q2': 'x'.join(map(str, q2)),
            'Computed M': computed,
            'Meter': meter
        })
    df = pd.DataFrame(rows)
    #display_dataframe_to_user('beatSync Results', df)
    print('beatSync Results')
    display(df)


# Execute the tests
test_beatSync()

In [5]:
def test_metricSync():
    # Trimmed rows for meters 2x2x2 and 2x3x2 with various v
    tests = [
        (1, [2,2,2,2], 1, [2,2,2,2], '4/4'),
        (1, [2,2,2,2], 1, [2,2,2,2], '4/4'),
        (1, [2,2,3,2], 1, [2,2,2,2], '12/8'),
        (1, [2,3,2,2], 1, [2,2,2,2], '6/4'),
        (1, [3,2,2], 1, [2,2,2,2], '3/4'),
        (1, [3,3,2,2], 1, [2,2,2,2], '9/4'),
        (1, [7,2,2], 1, [2,2,2,2], '7/4'),
        (1, [7,2], 1, [2,2,2,2], '7/8'),
    ]
    rows = []
    for v1, q1, v2, q2, meter in tests:
        m1 = {'q': q1, 'v': v1}
        m2 = {'q': q2, 'v': v2}
        computed = round(compute_affinity(m1, m2), 5)
        rows.append({
            'v1': v1,
            'q1': 'x'.join(map(str, q1)),
            'v2': v2,
            'q2': 'x'.join(map(str, q2)),
            'Computed M': computed,
            'Meter': meter
        })
    df = pd.DataFrame(rows)
    #display_dataframe_to_user('metricSync Results', df)
    print('metricSync Results')
    display(df)


# Execute the tests
test_metricSync()

metricSync Results


,v1,q1,v2,q2,Computed M,Meter
0,1,2x2x2x2,1,2x2x2x2,0.39318,4/4
1,1,2x2x2x2,1,2x2x2x2,0.39318,4/4
2,1,2x2x3x2,1,2x2x2x2,0.18045,12/8
3,1,2x3x2x2,1,2x2x2x2,0.15843,6/4
4,1,3x2x2,1,2x2x2x2,0.15083,3/4
5,1,3x3x2x2,1,2x2x2x2,0.09837,9/4
6,1,7x2x2,1,2x2x2x2,0.08717,7/4
7,1,7x2,1,2x2x2x2,0.08717,7/8
